# Install and load dependencies

In [1]:
# Install libraries
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install kaggle
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install tqdm

# Import libraries

## Progress bar
from tqdm import tqdm

## Data manipulation
import pandas as pd
import sqlite3

## Web scraping
import kaggle
from bs4 import BeautifulSoup
from pathlib import Path
import requests
import re
import zipfile
import io

/Users/jenspedermeldgaard/Library/Caches/pypoetry/virtualenvs/moonbetting-0mOBv6D2-py3.8/bin/python: No module named pip
/Users/jenspedermeldgaard/Library/Caches/pypoetry/virtualenvs/moonbetting-0mOBv6D2-py3.8/bin/python: No module named pip
/Users/jenspedermeldgaard/Library/Caches/pypoetry/virtualenvs/moonbetting-0mOBv6D2-py3.8/bin/python: No module named pip
/Users/jenspedermeldgaard/Library/Caches/pypoetry/virtualenvs/moonbetting-0mOBv6D2-py3.8/bin/python: No module named pip


# Define utility functions

In [2]:
def download_dataset(dataset_name: str, path: str):
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files(dataset_name, path=path, unzip=True)

def read_data(dir, name):
    df = pd.read_csv(f"{dir}/{name}.csv")
    print(f"{name}: {df.shape}")
    return df

def is_home_win(row):
    if row["home_club_goals"] > row["away_club_goals"]:
        return "1"
    elif row["home_club_goals"] < row["away_club_goals"]:
        return "2"
    else:
        return "x"



# Process Game Data

### Download data from Kaggle

In [3]:
download_dataset('davidcariboo/player-scores', 'data')

### Read data files and print shape

In [4]:
games = read_data('data/', "games")
clubs = read_data('data/' , "clubs")
player_valuations = read_data('data/', "player_valuations")


games: (60761, 21)
clubs: (411, 15)
player_valuations: (419678, 7)


### Set flag for winning team

In [5]:
games["winner"] = games.apply(is_home_win, axis=1)

### Add Club name to games

In [6]:
games = games.merge(clubs[['club_id', 'name']].rename(columns={"club_id": "home_club_id", 'name': 'home_club_name'}), on="home_club_id")
games = games.merge(clubs[['club_id', 'name']].rename(columns={"club_id": "away_club_id", 'name': 'away_club_name'}), on="away_club_id")

### Process Player Valuations

**Steps:**
1. Format player_valuations so that we know when a certain player had a certain value
2. Calculate value per club at the time of a specific game
3. Join club value onto games

##### 1. Format player valuations

In [7]:
sorted_players = player_valuations.sort_values(by=["player_id", "current_club_id", "date"]).rename(columns={"current_club_id": "club_id", "date": "start_date"})
sorted_players["end_date"] = sorted_players.groupby(["player_id", "club_id"])["start_date"].shift(-1)
sorted_players["end_date"] = sorted_players["end_date"].fillna(pd.to_datetime("today").date())
sorted_players

,start_date,datetime,dateweek,player_id,club_id,market_value_in_eur,player_club_domestic_competition_id,end_date
89289,2004-10-04,2004-10-04,2004-10-04,10,398,7000000,IT1,2005-01-07
89290,2005-01-07,2005-01-07,2005-01-03,10,398,9000000,IT1,2005-05-05
89291,2005-05-05,2005-05-05,2005-05-02,10,398,12000000,IT1,2005-09-30
89292,2005-09-30,2005-09-30,2005-09-26,10,398,15000000,IT1,2006-01-09
89293,2006-01-09,2006-01-09,2006-01-09,10,398,20000000,IT1,2006-07-15
...,...,...,...,...,...,...,...,...
356049,2022-08-01,2022-08-01,2022-08-01,1050852,48726,50000,UKR1,2022-12-06
356050,2022-12-06,2022-12-06,2022-12-05,1050852,48726,50000,UKR1,2023-03-02
378344,2022-11-04,2022-11-04,2022-10-31,1052404,354,50000,BE1,2023-03-02
410751,2022-11-01,2022-11-01,2022-10-31,1056907,2995,100000,PO1,2023-03-02


##### 2. Calculate club value in each game

In [8]:
conn = sqlite3.connect(":memory:")

games.to_sql("games", conn, if_exists="replace")
sorted_players.to_sql("players", conn, if_exists="replace")

home_query = """
SELECT
    games.game_id,
    SUM(players.market_value_in_eur) as home_value_in_eur
FROM
    games
JOIN players ON games.home_club_id = players.club_id
WHERE games.date BETWEEN players.start_date AND players.end_date
GROUP BY games.game_id
"""

away_query = """
SELECT
    games.game_id,
    SUM(players.market_value_in_eur) as away_value_in_eur
FROM
    games
JOIN players ON games.away_club_id = players.club_id
WHERE games.date BETWEEN players.start_date AND players.end_date
GROUP BY games.game_id
"""

home_value = pd.read_sql_query(home_query, conn)
away_value = pd.read_sql_query(away_query, conn)


##### 3. Join club value per game onto games

In [9]:
games_with_value = games.merge(home_value, on="game_id").merge(away_value, on="game_id")
games_with_value

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,away_club_manager_name,stadium,attendance,referee,url,winner,home_club_name,away_club_name,home_value_in_eur,away_value_in_eur
0,2229332,DFL,other,2012,Final,2012-08-12,27,16,2,1,...,Jürgen Klopp,Allianz Arena,69000,Michael Weiner,https://www.transfermarkt.co.uk/spielbericht/i...,1,Fc Bayern Munchen,Borussia Dortmund,178150000,83800000
1,2231446,L1,domestic_league,2012,15. Matchday,2012-12-01,27,16,1,1,...,Jürgen Klopp,Allianz Arena,71000,Peter Gagelmann,https://www.transfermarkt.co.uk/bayern-munich_...,x,Fc Bayern Munchen,Borussia Dortmund,175175000,93800000
2,2284437,DFB,domestic_cup,2012,Quarter-Finals,2013-02-27,27,16,1,0,...,Jürgen Klopp,Allianz Arena,71000,Knut Kircher,https://www.transfermarkt.co.uk/spielbericht/i...,1,Fc Bayern Munchen,Borussia Dortmund,178475000,98250000
3,2321316,L1,domestic_league,2013,30. Matchday,2014-04-12,27,16,0,3,...,Jürgen Klopp,Allianz Arena,71000,Felix Zwayer,https://www.transfermarkt.co.uk/bayern-munich_...,2,Fc Bayern Munchen,Borussia Dortmund,205450000,135075000
4,2460707,L1,domestic_league,2014,10. Matchday,2014-11-01,27,16,2,1,...,Jürgen Klopp,Allianz Arena,71137,Manuel Gräfe,https://www.transfermarkt.co.uk/bayern-munich_...,1,Fc Bayern Munchen,Borussia Dortmund,230075000,165150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48021,3716651,GR1,domestic_league,2021,18. Matchday,2022-01-15,3999,553,2,1,...,Dimitrios Spanos,"Stadio Lamia ""Athanasios Diakos""",476,Anastasios Papapetrou,https://www.transfermarkt.co.uk/pas-lamia-1964...,1,Pas Lamia 1964,Ionikos Nikeas,25325000,18025000
48022,3889126,GR1,domestic_league,2022,20. Matchday,2023-01-30,3999,553,0,2,...,Michalis Grigoriou,"Stadio Lamia ""Athanasios Diakos""",1053,Christos Vergetis,https://www.transfermarkt.co.uk/pas-lamia-1964...,2,Pas Lamia 1964,Ionikos Nikeas,21410000,13775000
48023,3716703,GR1,domestic_league,2021,21. Matchday,2022-02-02,60949,553,1,1,...,Dimitrios Spanos,Panthessaliko Stadio,131,Spyros Zabalas,https://www.transfermarkt.co.uk/volos-nps_ioni...,x,Volos Nps,Ionikos Nikeas,22375000,17875000
48024,3889017,GR1,domestic_league,2022,5. Matchday,2022-09-18,60949,553,2,0,...,Dimitrios Spanos,Panthessaliko Stadio,2400,Spyros Zabalas,https://www.transfermarkt.co.uk/volos-nps_ioni...,1,Volos Nps,Ionikos Nikeas,22975000,16875000


### Processed Game Data

In [10]:
games_with_value.head()

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,away_club_manager_name,stadium,attendance,referee,url,winner,home_club_name,away_club_name,home_value_in_eur,away_value_in_eur
0,2229332,DFL,other,2012,Final,2012-08-12,27,16,2,1,...,Jürgen Klopp,Allianz Arena,69000,Michael Weiner,https://www.transfermarkt.co.uk/spielbericht/i...,1,Fc Bayern Munchen,Borussia Dortmund,178150000,83800000
1,2231446,L1,domestic_league,2012,15. Matchday,2012-12-01,27,16,1,1,...,Jürgen Klopp,Allianz Arena,71000,Peter Gagelmann,https://www.transfermarkt.co.uk/bayern-munich_...,x,Fc Bayern Munchen,Borussia Dortmund,175175000,93800000
2,2284437,DFB,domestic_cup,2012,Quarter-Finals,2013-02-27,27,16,1,0,...,Jürgen Klopp,Allianz Arena,71000,Knut Kircher,https://www.transfermarkt.co.uk/spielbericht/i...,1,Fc Bayern Munchen,Borussia Dortmund,178475000,98250000
3,2321316,L1,domestic_league,2013,30. Matchday,2014-04-12,27,16,0,3,...,Jürgen Klopp,Allianz Arena,71000,Felix Zwayer,https://www.transfermarkt.co.uk/bayern-munich_...,2,Fc Bayern Munchen,Borussia Dortmund,205450000,135075000
4,2460707,L1,domestic_league,2014,10. Matchday,2014-11-01,27,16,2,1,...,Jürgen Klopp,Allianz Arena,71137,Manuel Gräfe,https://www.transfermarkt.co.uk/bayern-munich_...,1,Fc Bayern Munchen,Borussia Dortmund,230075000,165150000


# Process Betting Data

## Scrape betting data

Betting data is gathered from https://www.football-data.co.uk

To get more information about the collected data, review the [notes](https://www.football-data.co.uk/notes.txt)

In [11]:
base = "https://www.football-data.co.uk"

page = requests.get(f"{base}//downloadm.php")
soup = BeautifulSoup(page.content, "html.parser")
csv_files = soup.find("b", string="CSV").parent.findAll("a", href=re.compile(r'.*.zip$'))
pb = tqdm(csv_files)

for csv_file in pb:
    name = re.sub(r'\W', '-', csv_file.text)
    href = csv_file.get("href")
    pb.set_description(f"Downloading {name} from {href}")
    csv_zip = requests.get(f"{base}/{href}", stream=True)
    with zipfile.ZipFile(io.BytesIO(csv_zip.content)) as z:
        z.extractall(f"data/bettings/{name}")

### Parse scraped data

In [12]:
column_data = {k: [] for k in "Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA".split(",")}
files=[f for f in Path("data/bettings").glob("**/*.csv")]

column_data["file"] = []
dfs = []
pb = tqdm(files)

for file in pb:
    with open(file, "rb") as f:
        pb.set_description(f"Parsing {file}")

        # Read headers
        headers = f.readline()

        # Find the positions of the columns we want
        header_positions = {k: v for v, k in enumerate(headers.decode("latin1").split(",")) if k in column_data}

        # Read the rest of the file
        for line in f:
            # Split the line into columns
            line = line.decode("latin1").split(",")
            # Remove the trailing newline
            line[-1] = line[-1].strip()

            # Skip empty lines
            if len(line) == 0:
                continue

            # Get data for each column
            for col in column_data:
                # Default to None
                data = pd.NA

                # Special case for file
                if col == "file":
                    data = file
                # Otherwise, get the data from the line
                elif col in header_positions:
                    try:
                        data = line[header_positions[col]].strip()
                        if data == "":
                            data = pd.NA
                    except Exception:
                        print(f"Error reading column {col} from {file}")
                
                column_data[col].append(data)

print("Done reading files")

bettings = pd.DataFrame(column_data)
bettings = bettings.dropna(subset=["Div", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR"])
bettings["Date"] = pd.to_datetime(bettings["Date"], format="%d/%m/%y", errors="ignore")
bettings.sort_values(by=["Date", "Div"], inplace=True)
bettings

Parsing data/bettings/Season-1994-1995/SP1.csv:  44%|████▍     | 275/623 [00:02<00:03, 111.91it/s]

Error reading column IWH from data/bettings/Season-2001-2002/I2.csv
Error reading column IWD from data/bettings/Season-2001-2002/I2.csv
Error reading column IWA from data/bettings/Season-2001-2002/I2.csv
Error reading column WHH from data/bettings/Season-2001-2002/I2.csv
Error reading column WHD from data/bettings/Season-2001-2002/I2.csv
Error reading column IWH from data/bettings/Season-2001-2002/I2.csv
Error reading column IWD from data/bettings/Season-2001-2002/I2.csv
Error reading column IWA from data/bettings/Season-2001-2002/I2.csv
Error reading column WHH from data/bettings/Season-2001-2002/I2.csv
Error reading column WHD from data/bettings/Season-2001-2002/I2.csv
Error reading column IWH from data/bettings/Season-2001-2002/I2.csv
Error reading column IWD from data/bettings/Season-2001-2002/I2.csv
Error reading column IWA from data/bettings/Season-2001-2002/I2.csv
Error reading column WHH from data/bettings/Season-2001-2002/I2.csv
Error reading column WHD from data/bettings/Seas

Parsing data/bettings/Season-2006-2007/SC0.csv:  60%|█████▉    | 372/623 [00:03<00:02, 112.73it/s]

Error reading column Date from data/bettings/Season-2003-2004/SC2.csv
Error reading column HomeTeam from data/bettings/Season-2003-2004/SC2.csv
Error reading column AwayTeam from data/bettings/Season-2003-2004/SC2.csv
Error reading column FTHG from data/bettings/Season-2003-2004/SC2.csv
Error reading column FTAG from data/bettings/Season-2003-2004/SC2.csv
Error reading column FTR from data/bettings/Season-2003-2004/SC2.csv
Error reading column HTHG from data/bettings/Season-2003-2004/SC2.csv
Error reading column HTAG from data/bettings/Season-2003-2004/SC2.csv
Error reading column HTR from data/bettings/Season-2003-2004/SC2.csv
Error reading column B365H from data/bettings/Season-2003-2004/SC2.csv
Error reading column B365D from data/bettings/Season-2003-2004/SC2.csv
Error reading column B365A from data/bettings/Season-2003-2004/SC2.csv
Error reading column IWH from data/bettings/Season-2003-2004/SC2.csv
Error reading column IWD from data/bettings/Season-2003-2004/SC2.csv
Error reading

Parsing data/bettings/Season-2009-2010/E3.csv:  71%|███████▏  | 444/623 [00:04<00:01, 108.45it/s] 

...

Parsing data/bettings/Season-2015-2016/I1.csv:  83%|████████▎ | 517/623 [00:04<00:00, 115.56it/s] 

Error reading column IWH from data/bettings/Season-2000-2001/I2.csv
Error reading column IWD from data/bettings/Season-2000-2001/I2.csv
Error reading column IWA from data/bettings/Season-2000-2001/I2.csv
Error reading column WHH from data/bettings/Season-2000-2001/I2.csv
Error reading column WHD from data/bettings/Season-2000-2001/I2.csv
Error reading column IWH from data/bettings/Season-2000-2001/I2.csv
Error reading column IWD from data/bettings/Season-2000-2001/I2.csv
Error reading column IWA from data/bettings/Season-2000-2001/I2.csv
Error reading column WHH from data/bettings/Season-2000-2001/I2.csv
Error reading column WHD from data/bettings/Season-2000-2001/I2.csv
Error reading column IWH from data/bettings/Season-2000-2001/I2.csv
Error reading column IWD from data/bettings/Season-2000-2001/I2.csv
Error reading column IWA from data/bettings/Season-2000-2001/I2.csv
Error reading column WHH from data/bettings/Season-2000-2001/I2.csv
Error reading column WHD from data/bettings/Seas

Parsing data/bettings/Season-1998-1999/T1.csv: 100%|██████████| 623/623 [00:05<00:00, 105.86it/s] 


Done reading files


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,file
190303,E0,01/01/01,<NA>,Charlton,Arsenal,1,0,H,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-2000-2001/E0.csv
190304,E0,01/01/01,<NA>,Chelsea,Aston Villa,1,0,H,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-2000-2001/E0.csv
190305,E0,01/01/01,<NA>,Coventry,Man City,1,1,D,0,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-2000-2001/E0.csv
190306,E0,01/01/01,<NA>,Derby,Everton,1,0,H,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-2000-2001/E0.csv
190307,E0,01/01/01,<NA>,Leeds,Middlesbrough,1,1,D,0,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-2000-2001/E0.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99512,SC1,31/12/94,<NA>,Airdrie,Dunfermline,0,0,D,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-1994-1995/SC1.csv
99513,SC1,31/12/94,<NA>,Ayr,Hamilton,1,2,A,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-1994-1995/SC1.csv
99514,SC1,31/12/94,<NA>,Clydebank,Stranraer,2,3,A,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-1994-1995/SC1.csv
99515,SC1,31/12/94,<NA>,Dundee,St Mirren,4,0,H,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,data/bettings/Season-1994-1995/SC1.csv


### Check Data Consistency

In [13]:
games_with_value[games_with_value.home_club_name.str.contains("Arsenal")].home_club_name.unique()

array(['Fc Arsenal', 'Arsenal Tula', 'Arsenal Kiew'], dtype=object)

In [14]:
bettings[bettings.HomeTeam.str.contains("Arsenal")].HomeTeam.unique()

array(['Arsenal'], dtype=object)

### Conclusion

Unfortunately, team names differ between the free betting data and the games data. Hence, I don't find it realistic to join these two datasets as it would entail the creating of a mapping between the team names in each dataset. 

I would instead encourage the use of a paid API for the betting data.